# CS667: Practical Data Science - Fall 2025  
## A#2 Draft  
Authors: Michael Griffin, Ramakrishna Sonakam, Will Torres  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
df_complaints = pd.read_csv('/content/drive/MyDrive/PDS/data/complaints.csv')
df_stalled = pd.read_csv('/content/drive/MyDrive/PDS/data/stalled.csv')

## Inspect Data

In [ ]:
df_complaints.head()

In [ ]:
df_stalled.head()

In [ ]:

def inspect_dataframe(df):
    print("\nData types:")
    print(df.info())

    print("\nDescriptive statistics for numerical columns:")
    display(df.describe())

    # mg/rs - add more checks here...

In [ ]:
inspect_dataframe(df_complaints)

In [ ]:
inspect_dataframe(df_stalled)

In [ ]:
df_stalled['complaint_number'] = df_stalled['complaint_number'].astype('str')
df_complaints['complaint_number'] = df_complaints['complaint_number'].astype('str')

## Add Priority to Category

In [ ]:
complaint_priority_map = {
    "03":"A","04":"A","05":"B","06":"B","09":"B","10":"A","12":"A","13":"A",
    "14":"A","15":"A","16":"B","18":"A","20":"A","21":"B","23":"B","29":"B",
    "30":"C","31":"A","35":"C","37":"A","45":"B","49":"C","50":"A","52":"B",
    "53":"D","54":"B","55":"D","56":"A","58":"B","59":"B","62":"A","63":"B",
    "65":"A","66":"B","67":"A","71":"B","73":"C","74":"C","75":"B","76":"A",
    "77":"C","78":"B","79":"C","80":"D","81":"A","82":"A","83":"B","85":"C",
    "86":"A","88":"B","89":"A","90":"C","91":"A","92":"B","93":"B","94":"C",
    "1A":"B","1B":"B","1D":"B","1E":"A","1G":"B","1K":"D","1Z":"D","2A":"B",
    "2B":"A","2C":"B","2D":"B","2E":"A","2F":"D","2G":"C","2H":"D","2J":"D",
    "2K":"D","2L":"D","2M":"D","3A":"B","4A":"B","4B":"B","4C":"D","4D":"D",
    "4F":"D","4G":"B","4J":"D","4K":"D","4L":"D","4M":"D","4N":"D","4P":"C",
    "4W":"C","5A":"B","5B":"A","5C":"A","5E":"A","5F":"B","5G":"B","6A":"C"
}

df_complaints['priority'] = df_complaints['complaint_category'].map(complaint_priority_map)


### Reduce Dimensions of Disposition and Complaint Categories
- Label each feature to reduce into 'bins'

## Merging Datasets

In [ ]:
# Check for duplicate complaint numbers in each dataset
complaints_dupes = df_complaints['complaint_number'].duplicated().sum()
stalled_dupes = df_stalled['complaint_number'].duplicated().sum()

# Check unique complaint numbers
unique_complaints_df_complaints = df_complaints['complaint_number'].nunique()
unique_complaints_df_stalled = df_stalled['complaint_number'].nunique()

# Check for overlaping complaints
overlap = set(df_complaints['complaint_number']) & set(df_stalled['complaint_number'])
overlap_count = len(overlap)

print(f"Complaints Dataset: {len(df_complaints)} rows, {unique_complaints_df_complaints} unique complaints")
print(f"Stalled Dataset: {len(df_stalled)} rows, {unique_complaints_df_stalled} unique complaints")
print(f"Overlapping complaints: {overlap_count}")
print(f"Duplication rate - Complaints: {complaints_dupes/len(df_complaints):.2%}")
print(f"Duplication rate - Stalled: {stalled_dupes/len(df_stalled):.2%}")

In [ ]:
# Deduplicate datasets
df_complaints_clean = (df_complaints.sort_values('dobrundate', ascending=False)
             .drop_duplicates(subset='complaint_number', keep='first'))

df_stalled_clean = (df_stalled.sort_values('dobrundate', ascending=False)
             .drop_duplicates(subset='complaint_number', keep='first'))

print(f"Complaints after deduplication: {len(df_complaints_clean)} (removed {len(df_complaints) - len(df_complaints_clean)})")
print(f"Stalled after deduplication: {len(df_stalled_clean)} (removed {len(df_stalled) - len(df_stalled_clean)})")

In [ ]:
# IF WE ARE PRESERVING ALL DATA:
merged = df_complaints_clean.merge(
    df_stalled_clean[['complaint_number', 'date_complaint_received', 'dobrundate']],
    on='complaint_number',
    how='left',
    suffixes=('', '_stalled'),
    indicator=True  # to track source
)

# Create stalled indicator
merged['is_stalled'] = (merged['_merge'] == 'both').astype(int)

print(f"Total rows after merge: {len(merged)}")
print(merged['_merge'].value_counts())
# left_only = complaints without stalled status
# right_only = stalled sites without complaint details (data quality issue?)
# both = complete records

# Note: there is probably another/more efficient way to merge the data?

In [ ]:
merged.head(10)

In [ ]:
inspect_dataframe(merged)

In [ ]:
# Convert date columns
date_cols = ['date_entered', 'disposition_date', 'inspection_date', 'date_complaint_received']

for col in date_cols:
    if col in merged.columns:
        merged[col] = pd.to_datetime(merged[col], errors='coerce')

## Visualizations

In [ ]:
stalled_df = merged[merged['is_stalled'] == 1].copy()
stalled_df['date_complaint_received'] = pd.to_datetime(stalled_df['date_complaint_received'], errors='coerce')
stalled_df['year_month'] = stalled_df['date_complaint_received'].dt.to_period('M')
stalled_monthly_counts = stalled_df['year_month'].value_counts().sort_index()

plt.figure(figsize=(15, 7))
stalled_monthly_counts.plot(kind='line', marker='o', linestyle='-')
plt.title('Stalled Complaint Volume Over Time (Monthly)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=10)
plt.ylabel('Count', fontsize=10)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
'''
Visualizations:
- kde on map of zip_code
- disposition code
'''

In [ ]:
# bar of status
status_counts = merged['status'].value_counts()
plt.figure(figsize=(10, 6))
status_counts.plot(kind='bar')
plt.title('Status Counts')
plt.xlabel('Status')
plt.ylabel('Count')
plt.show()

print(f"Active complaints: {status_counts['ACTIVE']}")

In [ ]:
# top 10 complaint categories per priority label
priority_levels = ['A', 'B', 'C', 'D']

priority_colors = {
    'A': '#1f77b4',  # Blue
    'B': '#ff7f0e',  # Orange
    'C': '#2ca02c',  # Green
    'D': '#d62728'   # Red
}

complaints = merged.dropna(subset=['priority'])

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 10))
axes = axes.flatten()

for i, priority in enumerate(priority_levels):
    ax = axes[i]
    subset = complaints[complaints['priority'] == priority]

    top_categories = subset['complaint_category'].value_counts().nlargest(10).index
    top_subset = subset[subset['complaint_category'].isin(top_categories)]

    sns.countplot(
        data=top_subset,
        x='complaint_category',
        order=top_categories,
        ax=ax,
        color=priority_colors[priority]
    )

    ax.set_title(f'Top 10 Complaint Categories — Priority {priority}', fontsize=14)
    ax.set_xlabel('Complaint Category')
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Feature Transformation / Engineering

- **Classification** Idea: Can we flag construction sites likely to stall within the next 90 days?
  - Requires us to calculate a target feature based on historical data - i.e., determining sites that have stalled within 90D(?) of a given date
  - Engineered Features
    - `complaints_last_30_days` = Count of complaints in past 30 days  
    - `complaints_last_90_days` = Count of complaints in past 90 days  
    - `complaint_ROI` = complaints_last_30 / complaints_last_90 (rate of increase)  
    - `recent_priority_a_count` = Priority A complaints in last 60 days  
    - `days_since_last_complaint` = Current date - max(date_entered)  
    - `unresolved_complaints_current` = Count with no disposition as of prediction date  
    - `recent_stop_work_orders` = Stop work dispositions in last 90 days  
    - `inspection_frequency_recent` = Inspections in last 60 days  
    - `resolution_time_increasing` = Binary: avg resolution time last 60d > previous 60d
    - `total_complaints_to_date` = Count of all complaints up to prediction point  
    - `unique_categories_to_date` = Unique complaint categories to date  
    - `has_structural_complaints` = Ever had categories 12, 14, 30, 55, ...  
    - `has_permit_violations` = Ever had categories 05, 06, 66, ...
    - `has_illegal_work` = Ever had categories 76, 3A, 5G, ...
    - ... other metrics (maybe related to category patterns?)

- **Regression** Idea: Can we identify how long a stalled construction (in days) will remain inactive?
  - Transformed / Normalized Features
    - `date_entered` = datetime; extract: year, month, day_of_week, quarter  
    - `date_complaint_received` = datetime  
    - `disposition_date` = datetime  
    - `inspection_date` = datetime  
    - `complaint_category` = Map to priority level (A/B/C/D)  
    - `status` = Categorical encoding (Active/Closed/...other?)  
    - `disposition_code` = Group into outcome categories (violations/stop work/cleared)  
    - `borough` = Extract from complaint_number first digit  
    - `community_board` = Separate into borough_code and board_number
  - Engineered Features
    - **TARGET**: `stall_duration_days` = current_date - date_complaint_received (or last activity date?)  
    - `complaints_before_stall` = Count of complaints before stall date per BIN  
    - `time_to_stall` = date_complaint_received - min(date_entered) per BIN  
    - `avg_resolution_time_before_stall` = Mean(disposition_date - date_entered) per BIN  
    - `unresolved_complaints_at_stall` = Count where disposition_date is null at stall time  
    - `priority_a_ratio` = Priority A complaints / total complaints per BIN  
    - `complaint_category_diversity` = Number of unique complaint categories per BIN  
    - `stop_work_orders_issued` = Count of disposition codes indicating stop work  
    - `inspection_frequency` = Count of inspections / time active before stall  
    - `last_activity_days` = date_complaint_received - max(disposition_date, inspection_date)  
    - `violation_escalation_rate` = Change in complaint severity over time  
    - `seasonal_stall_indicator` = Month/quarter when stalled (construction seasonality)  
    - `days_first_to_last_complaint` = max(date_entered) - min(date_entered) per BIN  
    - `complaint_velocity` = complaints_before_stall / days_first_to_last_complaint



In [ ]:
# How do we want to embed/encode the data as it is mostly categorical/temporal?

In [ ]:
from sklearn.preprocessing import LabelEncoder as le
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# draft encoding
df = merged[[
    'status','zip_code','bin','community_board','complaint_category',
    'unit','disposition_date','disposition_code','inspection_date',
    'priority','is_stalled'
]].copy()

for col in ['disposition_date', 'inspection_date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = df[col].view('int64')

priority_map = {'A': 4, 'B': 3, 'C': 2, 'D': 1}
df['priority'] = df['priority'].map(priority_map)

df['community_board'] = df['community_board'].astype(str)
df['zip_code'] = df['zip_code'].astype(str)

cat_nominal = ['status','zip_code','community_board','complaint_category','unit','disposition_code']
df[cat_nominal] = df[cat_nominal].fillna('Unknown')

num_cols = ['bin','disposition_date','inspection_date','priority']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')
df[num_cols] = df[num_cols].fillna(-1)


In [ ]:
# for tree models -- ordinal/label-like encoding for nominal categoricals
tree_pre = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_nominal),
    ],
    remainder='passthrough'
)

# convert ord transformation into dataframe for debugging purposes
X_tree = tree_pre.fit_transform(df.drop(columns=['is_stalled']))
ord_cols = cat_nominal + [c for c in df.drop(columns=['is_stalled']) if c not in cat_nominal]
X_tree_df = pd.DataFrame(X_tree, columns=ord_cols)

In [ ]:
# correlation
df_corr = X_tree_df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(df_corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# for linear reg models -- One-Hot for nominal categoricals
lin_pre = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_nominal),
    ],
    remainder='passthrough'
)

# convert ohe transformation into dataframe for debugging purposes -- ### Note: CRASHES (USES TOO MUCH RAM)
X_linear = lin_pre.fit_transform(df.drop(columns=['is_stalled']))

ohe = lin_pre.named_transformers_['ohe']
ohe_cols = ohe.get_feature_names_out(cat_nominal)

num_cols = [c for c in df.drop(columns=['is_stalled']) if c not in cat_nominal]

lin_cols = list(ohe_cols) + num_cols
X_linear_df = pd.DataFrame(X_linear.toarray(), columns=lin_cols)

## Data Normalization/Standardization

## Modeling

Classification:
- Logistic, KNN, Naive Bayes
- RF, XGB, Light GB

Regression:
- Linear, ARIMA
- Lasso, Ridge, SVR (Polynomial, RBF) -- if applicable